# Feature Engineering

---

1. Import packages
2. Load data
3. Feature engineering

---

## 1. Import packages

In [1]:
import pandas as pd

---
## 2. Load data

In [2]:
df = pd.read_csv('./clean_data_after_eda.csv')
df["date_activ"] = pd.to_datetime(df["date_activ"], format='%Y-%m-%d')
df["date_end"] = pd.to_datetime(df["date_end"], format='%Y-%m-%d')
df["date_modif_prod"] = pd.to_datetime(df["date_modif_prod"], format='%Y-%m-%d')
df["date_renewal"] = pd.to_datetime(df["date_renewal"], format='%Y-%m-%d')

In [3]:
df.head(3)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.000131,4.100838e-05,0.000908,2.086294,99.530517,44.235794,2.086425,9.953056e+01,44.236702,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000003,1.217891e-03,0.000000,0.009482,0.000000,0.000000,0.009485,1.217891e-03,0.000000,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000004,9.450150e-08,0.000000,0.000000,0.000000,0.000000,0.000004,9.450150e-08,0.000000,0


---

## 3. Feature engineering

### Difference between off-peak prices in December and preceding January

Below is the code created by your colleague to calculate the feature described above. Use this code to re-create this feature and then think about ways to build on this feature to create features with a higher predictive power.

In [10]:
price_df = pd.read_csv('clean_data_after_eda.csv')
# Extracting useful date features from the existing date columns
date_columns = ['date_activ', 'date_end', 'date_modif_prod', 'date_renewal']

# Create new columns for year, month, and day for each date column
for col in date_columns:
    price_df[col + '_year'] = pd.to_datetime(price_df[col]).dt.year
    price_df[col + '_month'] = pd.to_datetime(price_df[col]).dt.month
    price_df[col + '_day'] = pd.to_datetime(price_df[col]).dt.day

# Display the first few rows to see the newly created columns
print(price_df.head())


                                 id                     channel_sales  \
0  24011ae4ebbe3035111d65fa7c15bc57  foosdfpfkusacimwkcsosbicdxkicaua   
1  d29c2c54acc38ff3c0614d0a653813dd                           MISSING   
2  764c75f661154dac3a6c254cd082ea7d  foosdfpfkusacimwkcsosbicdxkicaua   
3  bba03439a292a1e166f80264c16191cb  lmkebamcaaclubfxadlmueccxoimlema   
4  149d57cf92fc41cf94415803a877cb4b                           MISSING   

   cons_12m  cons_gas_12m  cons_last_month  date_activ    date_end  \
0         0         54946                0  2013-06-15  2016-06-15   
1      4660             0                0  2009-08-21  2016-08-30   
2       544             0                0  2010-04-16  2016-04-16   
3      1584             0                0  2010-03-30  2016-03-30   
4      4425             0              526  2010-01-13  2016-03-07   

  date_modif_prod date_renewal  forecast_cons_12m  ...  date_activ_day  \
0      2015-11-01   2015-06-23               0.00  ...            

In [15]:

required_columns = ['var_6m_price_off_peak_var', 'date_activ']  # Assuming there's no fixed price column
missing_columns = [col for col in required_columns if col not in price_df.columns]

if missing_columns:
    raise KeyError(f"Column(s) {missing_columns} do not exist in the dataframe")

# Convert 'date_activ' to datetime format
price_df['date_activ'] = pd.to_datetime(price_df['date_activ'], format='%Y-%m-%d')

# Group by id and date_activ to aggregate off-peak prices
monthly_price_by_id = price_df.groupby(['id', 'date_activ']).agg({
    'var_6m_price_off_peak_var': 'mean'  # Using var_6m_price_off_peak_var as the only price column
}).reset_index()

# Filter for December and January prices
dec_prices = monthly_price_by_id[monthly_price_by_id['date_activ'].dt.month == 12]
jan_prices = monthly_price_by_id[monthly_price_by_id['date_activ'].dt.month == 1]

# Rename columns for merging
dec_prices = dec_prices.rename(columns={'var_6m_price_off_peak_var': 'dec_1'})
jan_prices = jan_prices.rename(columns={'var_6m_price_off_peak_var': 'jan_1'})

# Merge December and January prices on 'id'
diff = pd.merge(dec_prices[['id', 'dec_1']], jan_prices[['id', 'jan_1']], on='id', how='inner')

# Calculate the difference between December and January off-peak prices
diff['offpeak_diff_dec_january'] = diff['dec_1'] - diff['jan_1']

# Select the relevant columns
diff = diff[['id', 'offpeak_diff_dec_january']]

# Display the result
diff.head()



,id,offpeak_diff_dec_january


Now it is time to get creative and to conduct some of your own feature engineering! Have fun with it, explore different ideas and try to create as many as yo can!

In [16]:
# Convert 'date_activ' to datetime
price_df['date_activ'] = pd.to_datetime(price_df['date_activ'], format='%Y-%m-%d')

# Extract date-based features
price_df['year'] = price_df['date_activ'].dt.year
price_df['month'] = price_df['date_activ'].dt.month
price_df['day'] = price_df['date_activ'].dt.day
price_df['day_of_week'] = price_df['date_activ'].dt.dayofweek
price_df['is_month_start'] = price_df['date_activ'].dt.is_month_start.astype(int)
price_df['is_month_end'] = price_df['date_activ'].dt.is_month_end.astype(int)

# Create rolling averages
price_df['rolling_avg_price'] = price_df['var_6m_price_off_peak_var'].rolling(window=3).mean()

# Lag feature for consumption forecast
price_df['forecast_lag_1'] = price_df['forecast_cons_12m'].shift(1)

# Create churn-related features
price_df['time_active'] = (pd.to_datetime('today') - price_df['date_activ']).dt.days

# Display the newly created features
price_df.head()

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,date_renewal_day,year,month,day,day_of_week,is_month_start,is_month_end,rolling_avg_price,forecast_lag_1,time_active
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,23,2013,6,15,5,0,0,NaN,NaN,4142
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,31,2009,8,21,4,0,0,NaN,0.00,5536
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,17,2010,4,16,4,0,0,0.000046,189.95,5298
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,...,31,2010,3,30,1,0,0,0.000003,47.96,5315
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,...,9,2010,1,13,2,0,0,0.000006,240.04,5391
